# Mapa Folium HeatMap

In [5]:
# heatmap con movimiento
import folium
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM
import pandas as pd
import pymongo

df=pd.read_csv('mex_earthquakes.csv')

df.head()

,_id,Fecha,Hora,Latitud,Longitud,Profundidad,Intensidad
0,"ObjectId(""5db7805d3771a8f231530f47"")",2019-10-28,09:08:45,26.04,-107.89,10.0 km,4.0
1,"ObjectId(""5db7805d3771a8f231530f49"")",2019-10-28,08:15:02,14.13,-93.34,19.0 km,4.1
2,"ObjectId(""5db7805d3771a8f231530f4b"")",2019-10-28,04:59:59,17.90,-95.08,120.0 km,3.8
3,"ObjectId(""5db7805d3771a8f231530f4d"")",2019-10-28,04:55:15,16.69,-99.42,7.0 km,3.4
4,"ObjectId(""5db7805d3771a8f231530f4f"")",2019-10-28,04:34:54,16.30,-98.25,13.0 km,3.3


In [ ]:
# transformar segun datos de tiempo

df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')

df['month'] = df.pickup_datetime.apply(lambda x: x.month)

df['week'] = df.pickup_datetime.apply(lambda x: x.week)

df['day'] = df.pickup_datetime.apply(lambda x: x.day)

df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)

In [20]:
# inicializa el mapa

def generateBaseMap(default_location=[23.634501, -102.552784], default_zoom_start=4.6):
    
    base_map = folium.Map(location=default_location, 
                          control_scale=True, 
                          zoom_start=default_zoom_start)
    
    return base_map

In [21]:
mapa4=generateBaseMap()

mapa4

In [ ]:
# se copia df y mes>4

df_copy = df[df.month>4].copy()
df_copy['count'] = 1

In [ ]:
# datos para heatmap

df_copy[['pickup_latitude', 
         'pickup_longitude', 
         'count']].groupby(['pickup_latitude', 
                            'pickup_longitude']).sum().sort_values('count', 
                                                                   ascending=False).head(10)

In [ ]:
base_map = generateBaseMap() # crea mapa

HM(data=df_copy[['pickup_latitude', 
                  'pickup_longitude', 
                  'count']].groupby(['pickup_latitude', 
                                     'pickup_longitude']).sum().reset_index().values.tolist(), 
    radius=8, 
        max_zoom=13).add_to(base_map) # pon los datos en heatmap

base_map.save('base_map.html')  # guarda

base_map   # representa

In [ ]:
df_hour_list = []
for hour in df_copy.hour.sort_values().unique():
    df_hour_list.append(df_copy.loc[df_copy.hour == hour, 
                                    ['pickup_latitude', 
                                     'pickup_longitude', 
                                     'count']].groupby(['pickup_latitude', 
                                                        'pickup_longitude']).sum().reset_index().values.tolist())

In [ ]:
base_map = generateBaseMap(default_zoom_start=11) # crea mapa

HMWT(df_hour_list, 
     radius=5, 
     gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
     min_opacity=0.5, 
     max_opacity=0.8, 
     use_local_extrema=True).add_to(base_map) # datos pata heatmap con tiempo


base_map  # representa

In [ ]:
base_map.save('mapa.html') # guarda